<a href="https://colab.research.google.com/github/looee1q/PracticalML/blob/6.4-feature-proccessing/%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обработка признаков

В этом домашнем задании вы будете решать задачу предсказания стоимости автомобилей по их различным характеристикам.

In [198]:
# !pip uninstall -y category-encoders

# !pip uninstall -y scikit-learn

# !pip install category-encoders==2.5

# !pip install scikit-learn==1.3.1

# !pip install pandas==1.5.3

In [199]:
import pandas as pd
import numpy as np

RANDOM_STATE = 42

In [200]:
df = pd.read_csv("https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/cars_prices.csv", decimal='.')

In [201]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


### Описание некоторых признаков

`symboling` - rating corresponds to the degree to which the auto is more risky than its price indicates (+3 more risk and -3 is pretty safe)  
`make` - car types (i.e. car brand)  
`fuel-type` - types of fuel (gas or diesel)  
`aspiration` - engine aspiration (standard or turbo)  
`num-of-doors` - numbers of doors (two or four)  
`body-style` - car body style (sedan or hachback)  
`drive-wheels` - which types of drive wheel (forward-fwd, reversed-rwd)  
`engine-location` - engine mounted location (front or back)  
`wheel-base` - расстояние между осями передних и задних колес  
`length` - car lenght  
`weight` - car weight  
`width` - car width  
`height` - car height  

In [202]:
df.shape
df.replace('?', np.nan, inplace=True)

In [203]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  164 non-null    object 
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       203 non-null    object 
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

## Заполнение пропусков

Пропуски в этом датасете обозначены как `?`

In [204]:
for c in df.columns:
    print(c, df[c].isna().sum())

symboling 0
normalized-losses 41
make 0
fuel-type 0
aspiration 0
num-of-doors 2
body-style 0
drive-wheels 0
engine-location 0
wheel-base 0
length 0
width 0
height 0
curb-weight 0
engine-type 0
num-of-cylinders 0
engine-size 0
fuel-system 0
bore 4
stroke 4
compression-ratio 0
horsepower 2
peak-rpm 2
city-mpg 0
highway-mpg 0
price 4


Удалите строки, для которых неизвестно значение price, так как это целевая переменная.

## Вопрос для Quiz

Сколько строк осталось в данных?

In [205]:
df = df[df['price'].notna()]
df.shape
# df.info()

(201, 26)

In [206]:
columns_from_obj_to_num = ["normalized-losses", "bore", "stroke", "horsepower", "peak-rpm", "price"]
for column in columns_from_obj_to_num:
  df[column] = pd.to_numeric(df[column], errors='coerce')

<ipython-input-206-d68a40fa3e47>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = pd.to_numeric(df[column], errors='coerce')


## Вопрос для Quiz

Чему равно среднее значение `peak-rpm` до заполнения пропусков? Ответ округлите до целого числа.

In [207]:
print(f"mean value of 'peak-rpm' before fillin na is: {df['peak-rpm'].mean()}")

mean value of 'peak-rpm' before fillin na is: 5117.587939698493


Заполните средним значением пропуски в столбцах для числовых признаков и самым популярным значением для категориальных признаков
* `num-of-doors`
* `bore`
* `stroke`
* `horsepower`
* `peak-rpm`

In [208]:
df.fillna({"num-of-doors": df["num-of-doors"].mode()[0]}, inplace=True)

numeric_columns_1 = ["bore", "stroke", "horsepower", "peak-rpm"]
for column in numeric_columns_1:
  df.fillna({column: df[column].mean()}, inplace=True)

<ipython-input-208-dde195cdd823>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna({"num-of-doors": df["num-of-doors"].mode()[0]}, inplace=True)
<ipython-input-208-dde195cdd823>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna({column: df[column].mean()}, inplace=True)


In [209]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          201 non-null    int64  
 1   normalized-losses  164 non-null    float64
 2   make               201 non-null    object 
 3   fuel-type          201 non-null    object 
 4   aspiration         201 non-null    object 
 5   num-of-doors       201 non-null    object 
 6   body-style         201 non-null    object 
 7   drive-wheels       201 non-null    object 
 8   engine-location    201 non-null    object 
 9   wheel-base         201 non-null    float64
 10  length             201 non-null    float64
 11  width              201 non-null    float64
 12  height             201 non-null    float64
 13  curb-weight        201 non-null    int64  
 14  engine-type        201 non-null    object 
 15  num-of-cylinders   201 non-null    object 
 16  engine-size        201 non

Пропуски в столбце `normalized-losses` предскажите при помощи линейной регрессии по признакам
`symboling`, `wheel-base`, `length`, `width`, `height`, `curb-weight`, `engine-size`, `compression-ratio`, `city-mpg`, `highway-mpg` и заполните их предсказаниями

In [210]:
df_with_norm_losses = df[df['normalized-losses'].notna()].copy()
df_without_norm_losses = df[df['normalized-losses'].isna()].copy()

X_norm_losses = df_with_norm_losses[["symboling", "wheel-base", "length", "width", "height", "curb-weight", "engine-size", "compression-ratio", "city-mpg", "highway-mpg"]]
y_norm_losses = df_with_norm_losses['normalized-losses']

X_norm_losses_to_found = df_without_norm_losses[["symboling", "wheel-base", "length", "width", "height", "curb-weight", "engine-size", "compression-ratio", "city-mpg", "highway-mpg"]]

## Вопрос для Quiz

Чему равно предсказание линейной регрессии на первом пропущенном значении? Ответ округлите до целого числа.

In [211]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
linear_regression.fit(X_norm_losses, y_norm_losses)
prediction_norm_losses = linear_regression.predict(X_norm_losses_to_found)
prediction_norm_losses = np.round(prediction_norm_losses).astype(int)
prediction_norm_losses

array([168, 168, 134, 150, 124, 137, 127, 138, 131, 114, 156, 176, 166,
        88, 123, 128, 162, 154, 133, 178, 180, 180, 131, 120, 144, 121,
       178, 158, 158, 159,  98, 155, 121, 137, 108,  95, 106])

In [212]:
mask = df['normalized-losses'].isna()

# Заменяем NaN значениями из prediction_norm_losses
df.loc[mask, 'normalized-losses'] = prediction_norm_losses[:mask.sum()]

In [213]:
df['normalized-losses']

,normalized-losses
0,168.0
1,168.0
2,134.0
3,164.0
4,164.0
...,...
200,95.0
201,95.0
202,95.0
203,95.0


## 2. Кодирование категориальных признаков

1. Закодируйте бинарные признаки `fuel-type`, `aspiration`, `num-of-doors`, `engine-location` каждый отдельной колонкой, состоящей из 0 и 1.
Единицей кодируйте самую частую категорию.

In [214]:
df.describe(include='object')

,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,engine-type,num-of-cylinders,fuel-system
count,201,201,201,201,201,201,201,201,201,201
unique,22,2,2,2,5,3,2,6,7,8
top,toyota,gas,std,four,sedan,fwd,front,ohc,four,mpfi
freq,32,181,165,115,94,118,198,145,157,92


In [215]:
binary_categoral_features = ["fuel-type", "aspiration", "num-of-doors", "engine-location"]
for feature in binary_categoral_features:
  mode = df[feature].mode()[0]
  df[feature] = df[feature].map(lambda x: 1 if x == mode else 0)

<ipython-input-215-ff351fec9228>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = df[feature].map(lambda x: 1 if x == mode else 0)
<ipython-input-215-ff351fec9228>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = df[feature].map(lambda x: 1 if x == mode else 0)
<ipython-input-215-ff351fec9228>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [216]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,168.0,alfa-romero,1,1,0,convertible,rwd,1,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495
1,3,168.0,alfa-romero,1,1,0,convertible,rwd,1,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500
2,1,134.0,alfa-romero,1,1,0,hatchback,rwd,1,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500
3,2,164.0,audi,1,1,1,sedan,fwd,1,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950
4,2,164.0,audi,1,1,1,sedan,4wd,1,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450


2. Вынесите в переменную `y` целевую переменную `price`, а все остальные колонки - в матрицу `X`.

Закодируйте признаки `make`, `body-style`, `engine-type`, `fuel-system` при помощи LeaveOneOutEncoder.

**Дальше все время работайте с объектами `X`, `y`.**

In [217]:
X = df.drop(columns=['price'])
y = df['price']

In [218]:
from category_encoders.leave_one_out import LeaveOneOutEncoder

leave_one_hot_encoding_categorial_features = ["make", "body-style", "engine-type", "fuel-system"]
leave_one_hot_encoder = LeaveOneOutEncoder(cols = leave_one_hot_encoding_categorial_features, random_state=RANDOM_STATE)
leave_one_hot_encoder.fit(X, y)
X = leave_one_hot_encoder.transform(X)

In [219]:
X.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg
0,3,168.0,15498.333333,1,1,0,21890.500000,rwd,1,88.6,...,four,130,17605.141304,3.47,2.68,9.0,111.0,5000.0,21,27
1,3,168.0,15498.333333,1,1,0,21890.500000,rwd,1,88.6,...,four,130,17605.141304,3.47,2.68,9.0,111.0,5000.0,21,27
2,1,134.0,15498.333333,1,1,0,9957.441176,rwd,1,94.5,...,six,152,17605.141304,2.68,3.47,9.0,154.0,5000.0,19,26
3,2,164.0,17859.166667,1,1,1,14459.755319,fwd,1,99.8,...,four,109,17605.141304,3.19,3.40,10.0,102.0,5500.0,24,30
4,2,164.0,17859.166667,1,1,1,14459.755319,4wd,1,99.4,...,five,136,17605.141304,3.19,3.40,8.0,115.0,5500.0,18,22


## Вопрос для Quiz

Чему равно среднее значение в столбце `body-style` после кодирования? Ответ округлите до целого числа.

In [220]:
X["body-style"].mean()

13207.129353233831

3. Закодируйте признак `drive-wheels` при помощи OHE из библиотеки category_encoders.

In [221]:
from category_encoders.one_hot import OneHotEncoder

one_hot_encoder = OneHotEncoder(cols = ["drive-wheels"])
X = one_hot_encoder.fit_transform(X)

/usr/local/lib/python3.11/dist-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():


In [222]:
X.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels_1,drive-wheels_2,drive-wheels_3,...,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg
0,3,168.0,15498.333333,1,1,0,21890.500000,1,0,0,...,four,130,17605.141304,3.47,2.68,9.0,111.0,5000.0,21,27
1,3,168.0,15498.333333,1,1,0,21890.500000,1,0,0,...,four,130,17605.141304,3.47,2.68,9.0,111.0,5000.0,21,27
2,1,134.0,15498.333333,1,1,0,9957.441176,1,0,0,...,six,152,17605.141304,2.68,3.47,9.0,154.0,5000.0,19,26
3,2,164.0,17859.166667,1,1,1,14459.755319,0,1,0,...,four,109,17605.141304,3.19,3.40,10.0,102.0,5500.0,24,30
4,2,164.0,17859.166667,1,1,1,14459.755319,0,0,1,...,five,136,17605.141304,3.19,3.40,8.0,115.0,5500.0,18,22


4. В столбце `num-of-cylinders` категории упорядочены по смыслу. Закодируйте их подряд идущими числами, начиная с 1, согласно смыслу.

Подряд идущими числами означает - 1, 2, 3 и так далее без пропусков.

In [223]:
X["num-of-cylinders"].value_counts()

,num-of-cylinders
four,157
six,24
five,10
two,4
eight,4
three,1
twelve,1


In [224]:
trasform_num_of_cylinders = {"four" : 4, "six" : 6, "five" : 5, "two" : 2, "eight" : 8, "three" : 3, "twelve" : 12}
X["num-of-cylinders"] = X["num-of-cylinders"].map(trasform_num_of_cylinders)

In [225]:
X["num-of-cylinders"].value_counts()

,num-of-cylinders
4,157
6,24
5,10
2,4
8,4
3,1
12,1


## Вопрос для Quiz

Сколько столбцов получилось в матрице `X`?

In [226]:
X.shape

(201, 27)

Разбейте данные на тренировочную и тестовую часть в пропорции 3 к 1, зафиксируйте random_state = 42.

In [227]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=RANDOM_STATE)

In [228]:
pd.DataFrame(data = X_train, columns = X.columns)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels_1,drive-wheels_2,drive-wheels_3,...,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg
178,3,197.0,9885.812500,1,1,0,9957.441176,1,0,0,...,6,171,17605.141304,3.27,3.35,9.3,161.000000,5200.00000,20,24
5,2,150.0,17859.166667,1,1,0,14459.755319,0,1,0,...,5,136,17605.141304,3.19,3.40,8.5,110.000000,5500.00000,19,25
130,0,98.0,9595.000000,1,1,1,12371.960000,0,1,0,...,4,132,17605.141304,3.46,3.90,8.7,103.396985,5117.58794,23,31
120,1,154.0,7963.428571,1,1,1,9957.441176,0,1,0,...,4,90,7433.203125,2.97,3.23,9.4,68.000000,5500.00000,31,38
76,2,161.0,9239.769231,1,1,0,9957.441176,0,1,0,...,4,92,7433.203125,2.97,3.23,9.4,68.000000,5500.00000,37,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,0,131.0,15489.090909,1,1,1,12371.960000,1,0,0,...,4,120,17605.141304,3.46,3.19,8.4,97.000000,5000.00000,19,24
15,0,127.0,26118.750000,1,1,1,14459.755319,1,0,0,...,6,209,17605.141304,3.62,3.39,8.0,182.000000,5400.00000,16,22
95,1,128.0,10415.666667,1,1,0,9957.441176,0,1,0,...,4,97,7433.203125,3.15,3.29,9.4,69.000000,5200.00000,31,37
183,2,122.0,10077.500000,1,1,0,14459.755319,0,1,0,...,4,109,17605.141304,3.19,3.40,9.0,85.000000,5250.00000,27,34


Масштабируйте данные при помощи MinMaxScaler.

Обучайте масштабирование на тренировочных данных, а потом примените и к трейну, и к тесту.

In [229]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(X_train, y_train)
X_train = min_max_scaler.transform(X_train)
X_test = min_max_scaler.transform(X_test)

In [230]:
pd.DataFrame(data = X_train, columns = X.columns)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels_1,drive-wheels_2,drive-wheels_3,...,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg
0,1.0,0.691099,0.135656,1.0,1.0,0.0,0.000000,1.0,0.0,0.0,...,0.4,0.415094,1.0,0.579365,0.609524,0.14375,0.528037,0.428571,0.205882,0.216216
1,0.8,0.445026,0.414513,1.0,1.0,0.0,0.367504,0.0,1.0,0.0,...,0.3,0.283019,1.0,0.515873,0.633333,0.09375,0.289720,0.551020,0.176471,0.243243
2,0.4,0.172775,0.125485,1.0,1.0,1.0,0.197087,0.0,1.0,0.0,...,0.2,0.267925,1.0,0.730159,0.871429,0.10625,0.258864,0.394934,0.294118,0.405405
3,0.6,0.465969,0.068423,1.0,1.0,1.0,0.000000,0.0,1.0,0.0,...,0.2,0.109434,0.0,0.341270,0.552381,0.15000,0.093458,0.551020,0.529412,0.594595
4,0.8,0.502618,0.113062,1.0,1.0,0.0,0.000000,0.0,1.0,0.0,...,0.2,0.116981,0.0,0.341270,0.552381,0.15000,0.093458,0.551020,0.705882,0.675676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.4,0.345550,0.331623,1.0,1.0,1.0,0.197087,1.0,0.0,0.0,...,0.2,0.222642,1.0,0.730159,0.533333,0.08750,0.228972,0.346939,0.176471,0.216216
146,0.4,0.324607,0.703380,1.0,1.0,1.0,0.367504,1.0,0.0,0.0,...,0.4,0.558491,1.0,0.857143,0.628571,0.06250,0.626168,0.510204,0.088235,0.162162
147,0.6,0.329843,0.154187,1.0,1.0,0.0,0.000000,0.0,1.0,0.0,...,0.2,0.135849,0.0,0.484127,0.580952,0.15000,0.098131,0.428571,0.529412,0.567568
148,0.8,0.298429,0.142360,1.0,1.0,0.0,0.367504,0.0,1.0,0.0,...,0.2,0.181132,1.0,0.515873,0.633333,0.12500,0.172897,0.448980,0.411765,0.486486


Обучите на тренировочных данных линейную регрессию, сделайте предсказание на тесте и вычислите значение $R^2$ на тестовых данных.

In [231]:
lin_regression = LinearRegression()
lin_regression.fit(X_train, y_train)
y_prediction = lin_regression.predict(X_test)

## Вопрос для Quiz

Чему равно значение $R^2$ на тестовых данных? Ответ округлите до сотых.

In [232]:
from sklearn.metrics import r2_score
r2_score(y_test, y_prediction)

0.9204254662163991